In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import http.client
import json
from pandas import json_normalize

In [8]:
#import PL data
PL_19_20 = pd.read_csv('E0_19_20.csv')
PL_20_21 = pd.read_csv('E0_20_21.csv')
PL_21_22 = pd.read_csv('E0_21_22.csv')

#import BL data
BL_19_20 = pd.read_csv('D1_19_20.csv')
BL_20_21 = pd.read_csv('D1_20_21.csv')
BL_21_22 = pd.read_csv('D1_21_22.csv')


BL_21_22

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,D1,13/08/2021,19:30,M'gladbach,Bayern Munich,1,1,D,1,1,...,3.01,0.75,1.99,1.94,1.99,1.93,2.10,1.96,1.98,1.88
1,D1,14/08/2021,14:30,Augsburg,Hoffenheim,0,4,A,0,1,...,2.30,0.25,1.97,1.96,1.98,1.94,2.02,1.99,1.96,1.91
2,D1,14/08/2021,14:30,Bielefeld,Freiburg,0,0,D,0,0,...,1.64,0.25,1.75,2.05,1.79,2.16,1.80,2.20,1.76,2.12
3,D1,14/08/2021,14:30,Stuttgart,Greuther Furth,5,1,H,2,0,...,2.16,-0.75,2.02,1.77,2.09,1.83,2.12,1.86,2.07,1.80
4,D1,14/08/2021,14:30,Union Berlin,Leverkusen,1,1,D,1,1,...,1.76,0.00,1.92,2.01,1.92,2.01,1.95,2.02,1.91,1.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,D1,13/03/2022,14:30,Leverkusen,FC Koln,0,1,A,0,0,...,2.57,-0.75,1.88,2.02,1.88,2.05,1.91,2.17,1.85,2.02
229,D1,13/03/2022,16:30,Dortmund,Bielefeld,1,0,H,1,0,...,2.56,-1.50,2.01,1.89,2.01,1.92,2.04,2.00,1.94,1.92
230,D1,13/03/2022,16:30,Ein Frankfurt,Bochum,2,1,H,0,1,...,2.19,-1.00,1.97,1.93,1.99,1.93,2.23,1.96,2.00,1.86
231,D1,13/03/2022,18:30,Greuther Furth,RB Leipzig,1,6,A,1,4,...,2.41,1.50,1.96,1.94,1.96,1.96,2.05,1.99,1.94,1.91


In [39]:
#API Import


conn = http.client.HTTPSConnection("api-football-v1.p.rapidapi.com")

headers = {
    'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
    'x-rapidapi-key': "b8bd6a409emshdd09020617fc3afp1d89f3jsnf6eb60efca6d"
    }

conn.request("GET", "/v3/fixtures/statistics?fixture=2755", headers=headers)

res = conn.getresponse()
data = res.read()

#res.encoding 'utf-8'
out = json.loads(data)
#data.decode("utf-8")
print (json.dumps(out, indent = 4, sort_keys=True))
#print(out)

{
    "errors": [],
    "get": "fixtures/statistics",
    "paging": {
        "current": 1,
        "total": 1
    },
    "parameters": {
        "fixture": "2755"
    },
    "response": [
        {
            "statistics": [
                {
                    "type": "Shots on Goal",
                    "value": 3
                },
                {
                    "type": "Shots off Goal",
                    "value": 6
                },
                {
                    "type": "Total Shots",
                    "value": 15
                },
                {
                    "type": "Blocked Shots",
                    "value": 6
                },
                {
                    "type": "Shots insidebox",
                    "value": 8
                },
                {
                    "type": "Shots outsidebox",
                    "value": 7
                },
                {
                    "type": "Fouls",
                    "value": 18
   